<h1>Personal project to read and clasify bills</h1>

<li>Rossum API (Software to read invoices)</li>

<li>maximilianovedoya@gmail.com M@xi200122</li>


<h3>Read invoces using OCR</h3>
<li>it is necessary to read pdf </li>
<li>it is necessary to be able to extract invoices from nested directories</li>

In [237]:
import string
import pytesseract
from PIL import Image
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files (x86)\Tesseract-OCR\tesseract.exe'

import os
from os import listdir
from os.path import isfile, join

#temporary fix
if 'TestFiles' in os.getcwd():
    os.chdir('..')

files = [f for f in listdir('TestFiles') if isfile(join('TestFiles', f))]
data=list()
os.chdir('TestFiles')

for file in files:
    img=Image.open(file)
    text=pytesseract.image_to_string(img,lang='spa')
    striptext = text.replace('\n\n', ' ')
    striptext = striptext.replace('\n', ' ')
    striptext = striptext.replace(':', ' ')
    striptext=striptext.lower()
    data.append(striptext)

<h3>Build function to extract informatio from invoices.</h3>
<li>it is necessary to Improve the regex expresions</li>

In [433]:
def extratcor(striptext):
    import re 
    data={ 'iva':None,
           'fecha':None,
           'total':None,
           'empresa':None,
           'keywords':None,
          }

    keywords=['encomiendas','pasajes','boletos','latex','pincel',]

#it is necessary to improve this 
    iva=r'21,00\s*%|21\s*%|iva.{1,4}exento|10,50\s*%'
    iva=re.search(iva,striptext)
    if iva:
        data['iva']=iva.group(0)
    
#     importe=r'total\s*\$?\s+(\d+,?.?\d+.?\d{2})'
    importe=r'total\s*\$?\s+(\d{1,3}?\,?\d{1,3}?\,?\d{1,3}\.\d{2})'
    importe=re.search(importe,striptext)
    if importe:
        data['total']=importe.group(1)
    else:
        importe=r'total\s*\$?\s+(\d{1,3}?\.?\d{1,3}?\.?\d{1,3}\,\d{2})'
        importe=re.search(importe,striptext)
        if importe:
            data['total']=importe.group(1)
    
    fecha=r'\s+(\d{2}/\d{2}/\d{2})'
    fecha=re.search(fecha,striptext)
    if fecha:
        data['fecha']=fecha.group(1)
    else:
        fecha=r'\s+(\d{2}-\d{2}-\d{2})'
        fecha=re.search(fecha,striptext)
        data['fecha']=fecha.group(1)
    
    tipos=['s.r.l','srl','s.a','sa','s.a.s','sas']
    for tipo in tipos:
        empresa=r'(\w+\s*\w*\s*\w*\s*\w*)\s+'+tipo
        empresa=re.search(empresa,striptext)
        if empresa:
            data['empresa']=empresa.group(0)
    
    import string
    from nltk import word_tokenize
    text=word_tokenize(striptext)
    clean=[word for word in text if word not in nltk.corpus.stopwords.words("spanish")]
    clean=[word for word in clean if word not in string.punctuation]
    
# matches against keywords or fuzzzy keywords inside the text.
    from fuzzywuzzy import fuzz
    temp_=list()
    for word in clean:
        for keyword in keywords:
            if fuzz.ratio(word, keyword)>=70:
                temp_.append(keyword)
    if temp_:
        data['keywords']=temp_
    
    return data

In [434]:
temp_=list()
for bill in data:
    temp_.append(extratcor(bill))

In [435]:
import pandas as pd 
df=pd.DataFrame(temp_)
df

,iva,fecha,total,empresa,keywords
0,"10,50%",23/08/20,"3.089,05",bo sa,[encomiendas]
1,iva exento,11/07/20,270.00,2018 sa,None
2,None,10-02-20,6124.05,i avenida s.a,None
3,None,13-02-20,None,i avenida s.a,None
4,None,14-02-20,1497.67,factura avenida s.a,None
5,None,10/03/20,"3,960.00",00073983 axoft argentina s.a,[encomiendas]
6,None,25/03/20,None,descripción sa,None
7,None,17/01/20,742.98,avenida sa,None
8,None,05-10-20,None,None,None
9,21%,29/02/20,"18,341.24",empresa rio uruguay s.r.l,[encomiendas]


<h3>Build a list of stopwords to clean the text get keywords.</h3>
<li>It is necessary to check the spelling of words and include more invoices so it becomes better</li>

In [436]:
textos=''
for item in data:
    textos+=item

#Takes a word and strip all non-alphabetic char  
def clean_word(word):
    temp_=''
    for item in word:
        if item.isalpha():
            temp_+=item 
    return temp_    

#takes an string, a returns a list of potential keywords 
def clean_numbers(textos):
    import string
    from nltk import word_tokenize
    striptext = textos.replace('\n\n', ' ')
    striptext = striptext.replace('\n', ' ')
    text=word_tokenize(striptext,'spanish')
    clean=[word for word in text if word not in nltk.corpus.stopwords.words("spanish")]
    clean=[word for word in clean if word not in string.punctuation] 
    for item in range(len(clean)):
        clean[item]=clean_word(clean[item])
    clean=[word for word in clean if not word.isnumeric()]
    return clean

#takes a list of words form texts and returns a list of stopwords, words that are not keywords
def nokeywords(list_):
    from nltk.probability import FreqDist
    list_=FreqDist(list_)
    stopwords=list()
    for word,freq in list_.most_common(100000):
        if freq>1:
            stopwords.append(word)
    return stopwords

stopwords=nokeywords(clean_numbers(textos))
keywords=[word for word in clean_numbers(data[6]) if word not in stopwords]
keywords=[word for word in keywords if len(word)>4 and len(word)<=13 ]


# keywords

In [409]:
def test(striptext):
    import re 
    data={ 'iva':None,
           'fecha':None,
           'total':None,
           'empresa':None,
           'keywords':None,
          }

    keywords=['encomiendas','pasajes','boletos','latex','pincel']
    
    iva=r'21,00\s*%|21\s*%|iva.{1,4}exento|10,50\s*%'
    iva=re.search(iva,striptext)
    if iva:
        data['iva']=iva.group(0)
    
    importe=r'total\s*\$?\s+(\d{1,3}?\,?\d{1,3}?\,?\d{1,3}\.\d{2})'
    importe=re.search(importe,striptext)
    if importe:
        data['total']=importe.group(1)
    else:
        importe=r'total\s*\$?\s+(\d{1,3}?\.?\d{1,3}?\.?\d{1,3}\,\d{2})'
        importe=re.search(importe,striptext)
        if importe:
            data['total']=importe.group(1)

    fecha=r'\s+(\d{2}/\d{2}/\d{2})'
    fecha=re.search(fecha,striptext)
    if fecha:
        data['fecha']=fecha.group(1)
    else:
        fecha=r'\s+(\d{2}-\d{2}-\d{2})'
        fecha=re.search(fecha,striptext)
        data['fecha']=fecha.group(1)

    tipos=['s.r.l','srl','s.a','sa','s.a.s','sas']
    for tipo in tipos:
        empresa=r'(\w+\s*\w*\s*\w*\s*\w*)\s+'+tipo
        empresa=re.search(empresa,striptext)
        if empresa:
            data['empresa']=empresa.group(0)
    
    import string
    from nltk import word_tokenize
    text=word_tokenize(striptext)
    clean=[word for word in text if word not in nltk.corpus.stopwords.words("spanish")]
    clean=[word for word in clean if word not in string.punctuation]
    
    #matches against keywords or fuzzzy keywords inside the text.
    from fuzzywuzzy import fuzz
    temp_=list()
    for word in clean:
        for keyword in keywords:
            if fuzz.ratio(word, keyword)>70:
                temp_.append(keyword)
    if temp_:
        data['keywords']=temp_
    
    return data